In [1]:
import os 
os.getcwd()

'C:\\Users\\32831\\Downloads\\python-master'

<imag src = "thread.png">

<img src="thread.png">

In [4]:

import multiprocessing

print("Number of cpu : ", multiprocessing.cpu_count())


Number of cpu :  8


In [5]:
#!/usr/bin/env python
from multiprocessing.pool import ThreadPool
from time import time as timer
import urllib.request

urls = ["https://yahoo.com", "http://google.com","http://ibm.com", "http://apple.com"]

def fetch_url(url):
    try:
        response = urllib.request.urlopen(url)
        return url, response.read(), None
    except Exception as e:
        return url, None, e

start = timer()
results = ThreadPool(20).imap_unordered(fetch_url, urls)
for url, html, error in results:
    if error is None:
        print("%r fetched in %ss" % (url, timer() - start))
    else:
        print("error fetching %r: %s" % (url, error))
print("Elapsed Time: %s" % (timer() - start,))

'http://google.com' fetched in 0.6175274848937988s
'http://apple.com' fetched in 1.7305123805999756s
'https://yahoo.com' fetched in 4.989057540893555s
'http://ibm.com' fetched in 5.263003349304199s
Elapsed Time: 5.263003349304199


In [9]:
import multiprocessing
def calc_square(number):
    print('Square:' , number * number)
    result = number * number
    print(result)
def calc_quad(number):
    print('Quad:' , number * number * number * number)
if __name__ == "__main__":
    number = 7
    #result = None
    p1 = multiprocessing.Process(target=calc_square, args=(number,))
    print(p1)
    p2 = multiprocessing.Process(target=calc_quad, args=(number,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    
    # Wont print because processes run using their own memory location                     
    #print(result)

<Process(Process-3, initial)>
